# Libs

In [1]:
!wget "https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/kw_to_kw_id.json"
!wget "https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/recipe_id_to_kw_id.json"
!wget "https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/mappers.json"
!wget "https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/image_id_to_recipe_id.json"

--2024-12-03 23:40:58--  https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/kw_to_kw_id.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-12-03 23:40:58 ERROR 404: Not Found.

--2024-12-03 23:40:58--  https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/recipe_id_to_kw_id.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-12-03 23:40:58 ERROR 404: Not Found.

--2024-12-03 23:40:58--  https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/mappers.json
Resol

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [25]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
import pickle as pkl
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Dataset

In [4]:
reviews_train = pd.read_csv("https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/review_train.csv")
reviews_test = pd.read_csv("https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/review_test.csv")
recipes = pd.read_csv("https://raw.githubusercontent.com/Lord-Kelsier/datasets/refs/heads/master/recipes.csv")

recipes.set_index("RecipeId", inplace=True)

In [5]:
nutrition_facts = [
    'Calories', 'FatContent','SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
    'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent'
]

In [6]:
recipes["RecipeIngredientParts"] = recipes["RecipeIngredientParts"].apply(lambda x:  x.strip("[]").split(","))
recipes["RecipeIngredientParts"] = recipes["RecipeIngredientParts"].apply(lambda x: [] if len(x) == 1 and x[0] == "" else x)
recipes["RecipeIngredientParts"] = recipes["RecipeIngredientParts"].apply(lambda x: [int(i) for i in x])


In [7]:
all_ingredients = set()
list(map(lambda x: all_ingredients.update(x), recipes["RecipeIngredientParts"]))
len(all_ingredients)

6157

# Metrics

In [8]:
from dataclasses import dataclass

@dataclass
class Recipe:
    id: int
    rating: float

@dataclass
class UserRecomedation:
  user_id: int
  top_n: list[Recipe]

def precision_at_k(r, k):
    assert 1 <= k <= r.size
    return (np.asarray(r)[:k] != 0).mean()

def average_precision_at_k(r, k):
    r = np.asarray(r)
    score = 0.
    for i in range(min(k, r.size)):
        score += precision_at_k(r, i + 1)
    return score / k
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def idcg_at_k(k):
    return dcg_at_k(np.ones(k), k)

def ndcg_at_k(r, k, max_relevant):
    idcg = idcg_at_k(min(k, max_relevant))
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

def recall_at_k(relevant_items, recommended_items, k):
    relevant_items = set(relevant_items)
    recommended_items = set(recommended_items[:k])
    intersection = relevant_items.intersection(recommended_items)
    recall = len(intersection) / len(relevant_items) if len(relevant_items) > 0 else 0
    return recall

In [39]:
def novelty(recommended_items, items_popularity):
    novelty_score = 0
    for item in recommended_items:
        novelty_score += np.log(1 / items_popularity[item])
    return novelty_score

def diversity(recommended_items, item_categories):
    unique_categories = set()
    for item in recommended_items:
        categories = item_categories[item]
        if type(categories) == list:
            unique_categories.update(categories)
        else:
            unique_categories.add(categories)
    return len(unique_categories) / len(recommended_items)
def evaluate_ranking(recomendations, user_items_test, k, items_popularity, item_categories):
    # mean_ap = 0. # o MAP
    mean_precision = 0.
    mean_ndcg = 0.
    mean_recall = 0.
    mean_novelty = 0.
    mean_diversity = 0

    for u in tqdm(user_items_test.keys()):
        rec = recomendations[u]

        # checkeamos si los items son relevantes, i.e. estan evaluados (quizas con una nota > 3) ( esto no se hace)
        rel_vector = np.isin(rec, user_items_test[u], assume_unique=True).astype(int)
        mean_novelty += novelty(rec, items_popularity)
        # mean_ap += average_precision_at_k(rel_vector, k)
        mean_precision += precision_at_k(rel_vector, k)
        mean_ndcg += ndcg_at_k(rel_vector, k, len(user_items_test[u]))
        mean_recall += recall_at_k(user_items_test[u], rec, k)
        mean_diversity += diversity(rec, item_categories)

    mean_diversity /= len(user_items_test)
    mean_novelty /= len(user_items_test)
    mean_precision /= len(user_items_test)
    # mean_ap /= len(user_items_test)
    mean_ndcg /= len(user_items_test)
    mean_recall /= len(user_items_test)

    return mean_precision, mean_ndcg, mean_recall, mean_novelty, mean_diversity



In [10]:
items_counts = reviews_train["RecipeId"].value_counts()
items_popularity = items_counts / len(reviews_train)
items_popularity = items_popularity.to_dict()


In [11]:
all_recipes = list(set(recipes.index))

In [29]:
with open("mappers.json", "r") as f:
    mappers = json.load(f)
with open("recipe_id_to_kw_id.json", "r") as f:
    recipe_id_to_kw_id = json.load(f)

In [32]:
mappers['recipe_id_mapper']

current_id_to_original_id = {v:int(k) for k,v in mappers['recipe_id_mapper'].items()}
recipe_id_to_kw_id = {int(float(k)):v for k,v in recipe_id_to_kw_id.items()}

In [33]:
item_categories = {current_id: recipe_id_to_kw_id[old_id] for current_id, old_id in current_id_to_original_id.items()}

In [17]:
np.random.seed(314159265)
print(len(reviews_test["AuthorId"].unique()))

positive_test_set = reviews_test[reviews_test["Rating"] >= 5]
many_reviews_set = positive_test_set.groupby("AuthorId")["RecipeId"].count()
many_reviews_set = many_reviews_set[many_reviews_set >= 10]

test_users = set(many_reviews_set.index)
sample_users = np.random.choice(list(test_users), round(len(test_users) * 0.3))
reviews_test_sample = positive_test_set[positive_test_set["AuthorId"].isin(sample_users)]
test_recipes = list(set(reviews_test_sample["RecipeId"].unique()))
len(test_recipes), len(sample_users)

43925


(27705, 1165)

In [47]:
with open("basic_features_recomendations.pkl", "rb") as f:
  user_recomedations = pkl.load(f)


In [48]:
recomendations = {rec.user_id: [it.id for it in rec.top_n] for rec in user_recomedations}

In [49]:
sub_set_reviews_test = reviews_test_sample[reviews_test_sample["AuthorId"].isin(recomendations.keys())]
user_item_validation = sub_set_reviews_test.groupby("AuthorId")["RecipeId"].apply(list).to_dict()
precision, ndcg, recall, novelty_val, diversity_val = evaluate_ranking(
    recomendations,
    user_item_validation,
    10,
    items_popularity,
    item_categories
)
print(f"Precision: {precision}")
print(f"NDCG: {ndcg}")
print(f"Recall: {recall}")
print(f"Novelty: {novelty_val}")
print(f"Diversity: {diversity_val}")

  0%|          | 0/1018 [00:00<?, ?it/s]

Precision: 0.0017681728880157177
NDCG: 0.0019468872980914766
Recall: 0.00026988809208329344
Novelty: 117.58637467748132
Diversity: 3.467288801571711
